<center>
    <tr>
    <td><img src="ontario-tech-univ-logo.png" width="25%"></img></td>
    </tr>
</center>

# Image stitching

Faisal Qureshi   
Professor    
Faculty of Science    
Ontario Tech University    
Oshawa ON Canada    
http://vclab.science.ontariotechu.ca

## Copyright information
© Faisal Qureshi

## License

This work is licensed under a [Creative Commons Attribution-NonCommercial 4.0 International License.](https://creativecommons.org/licenses/by-nc/4.0/)

## Outline
Consider the following image pair

<center>
    <tr>
    <td>
    <img src="1-left.jpeg" width="30%"></img>
    <img src="1-right.jpeg" width="30%"></img>
    </td>
    </tr>
</center>

You are asked to pick at least four correspondences in both images. E.g., you can pick four points  $(𝑥^𝑙_{𝑖},𝑦^𝑙_{𝑖})$  in the left image and then you can pick the corresponding locations  $(𝑥^𝑟_{𝑖},𝑦^𝑟{𝑖})$  in the right image in the same order. Here  𝑖 > 3 . Given these correspondences, estimate the homography matrix between the two set of images, and use this matrix to stitch the two images into a single image.

## Picking locations in an image
Use the following code to pick locations on an image as seen below

![Paper written in LaTeX](image.png)

Modify the code as needed to record the  (𝑥,𝑦)  locations.

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
import numpy as np

In [2]:
class PointPicker:
    points_selected = []
    
    def __init__(self, imgplot, img, color=(255,0,0), radius=20, thickness=20):
        self.imgplot = imgplot
        self.img = img
        self.color = color
        self.radius = radius
        self.thickness = thickness
        self.cid = imgplot.figure.canvas.mpl_connect('button_press_event', self)
    
    def __call__(self, event):
        if event.inaxes != self.imgplot.axes: 
            return
        ix = event.xdata
        iy = event.ydata
        print(f'x={ix}, y={iy}')
        self.img = cv.circle(self.img, (int(ix), int(iy)), self.radius, self.color, self.thickness)
        
        imgplot.set_array(self.img)
        self.imgplot.figure.canvas.draw()
    
    def clear_points(self):
        self.points_selected.clear()

img=cv.imread('data/1-left.jpeg')        
        
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
imgplot = plt.imshow(img)
point_picker = PointPicker(imgplot, img)
plt.show()

In [8]:
def match_descriptors(img1, img2):
    akaze = cv.AKAZE_create()
    kp1, des1 = akaze.detectAndCompute(img1, None)
    kp2, des2 = akaze.detectAndCompute(img2, None)
    
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE_HAMMING)
    nn_matches = matcher.knnMatch(des1, des2, 2)
    
    good = []
    src = []
    dst = []
    for match in nn_matches:
        if len(match) < 2: continue
        m, n = match[0], match[1]
        if m.distance < 0.8*n.distance:
            good.append([m])
            src.append(kp1[m.queryIdx].pt)
            dst.append(kp2[n.trainIdx].pt)

    result_img = cv.drawMatchesKnn(img1,kp1,img2,kp2, good, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return np.float32(src), np.float32(dst), result_img

In [13]:
right_img = cv.imread("data/1-right.jpeg") 
left_img = cv.imread("data/1-left.jpeg")

src, dst, result_img = match_descriptors(right_img, left_img)
    

H, masked = cv.findHomography(src, dst, cv.RANSAC, 5.0)

dst = cv.warpPerspective(right_img,H,(left_img.shape[1] + right_img.shape[1], left_img.shape[0]))
dst[0:left_img.shape[0], 0:left_img.shape[1]] = left_img
cv.imwrite("data/1-stitched.jpg",dst)
plt.figure(figsize=(15,15))
plt.imshow(dst)
plt.show()

#This didn't yield desired results

In [10]:
right_img = cv.imread("data/2-right.jpeg") 
left_img = cv.imread("data/2-left.jpeg")

src, dst, result_img = match_descriptors(right_img, left_img)
    

H, masked = cv.findHomography(src, dst, cv.RANSAC, 5.0)

dst = cv.warpPerspective(right_img,H,(left_img.shape[1] + right_img.shape[1], left_img.shape[0]))
dst[0:left_img.shape[0], 0:left_img.shape[1]] = left_img
cv.imwrite("data/2-stitched.jpg",dst)
plt.imshow(dst)
plt.imshow()

#This did yield desired results

In [ ]:
right_img = cv.imread("data/3-right.jpeg") 
left_img = cv.imread("data/3-left.jpeg")

src, dst, result_img = match_descriptors(right_img, left_img)
    

H, masked = cv.findHomography(src, dst, cv.RANSAC, 5.0)

dst = cv.warpPerspective(right_img,H,(left_img.shape[1] + right_img.shape[1], left_img.shape[0]))
dst[0:left_img.shape[0], 0:left_img.shape[1]] = left_img
cv.imwrite("data/3-stitched.jpg",dst)
plt.imshow(dst)
plt.imshow()

#This did yield desired results

In [ ]:
right_img = cv.imread("data/4-right.jpeg") 
left_img = cv.imread("data/4-left.jpeg")

src, dst, result_img = match_descriptors(right_img, left_img)
    

H, masked = cv.findHomography(src, dst, cv.RANSAC, 5.0)

dst = cv.warpPerspective(right_img,H,(left_img.shape[1] + right_img.shape[1], left_img.shape[0]))
dst[0:left_img.shape[0], 0:left_img.shape[1]] = left_img
cv.imwrite("data/4-stitched.jpg",dst)
plt.imshow(dst)
plt.imshow()

#This did yield the most desired results

## Task
 * Load left image (```data/1-left.jpeg```) and pick at least four points in order.
 * Load right image (```data/1-right.jpeg```) and pick the corresponding locations.
 * Use the correspondences to estimate homography.
 * Use the homography to stitch the images together.
 
### Some outdoor photography
 * The above image exhibits strong parallex. You may consider going for a walk and taking an image pair of your favorite neighbourhood landscape. Try not to translate the camera, just rotate in place.

### Useful information
 * [Least squares fitting](http://csundergrad.science.uoit.ca/courses/cv-notes/notebooks/15-least-squares.html)
 * [RANSAC](http://csundergrad.science.uoit.ca/courses/cv-notes/notebooks/17-ransac.html)
 * [Homography](http://csundergrad.science.uoit.ca/courses/cv-notes/notebooks/19-homography.html)
 * [Image sampling](http://csundergrad.science.uoit.ca/courses/cv-notes/notebooks/10-image-sampling.html)

### Bonus
 * Try stitching on different image pairs.
 * Can you stich more than one images to create a panorama?

## Submission
Include code and stiched image single jupyter notebook. Submit via canvas.

<center>
    <tr>
    <td><img src="ontario-tech-univ-logo.png" width="25%"></img></td>
    </tr>
</center>